<a href="https://colab.research.google.com/github/Rakshikags/CognifyzInternshipTasks/blob/main/Cognifyz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install scikit-learn

In [2]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report
from sklearn.neighbors import NearestNeighbors
import folium



In [5]:
# Load dataset
file_path = "/content/Dataset .csv"
df = pd.read_csv(file_path)

In [6]:
# Define categorical and numerical columns
categorical_cols = ['Has Table booking', 'Has Online delivery', 'Is delivering now', 'Rating text']
numerical_cols = ['Average Cost for two', 'Price range', 'Votes']

# One-hot encode categorical columns (excluding 'Cuisines' for now)
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
categorical_encoded = ohe.fit_transform(df[categorical_cols].astype(str))
categorical_encoded_df = pd.DataFrame(categorical_encoded, columns=ohe.get_feature_names_out())
df = df.drop(columns=categorical_cols).join(categorical_encoded_df)

# Label encode 'Cuisines' separately (for Task 3 and interpretability in Task 2)
cuisine_encoder = LabelEncoder()
df['Cuisines_encoded'] = cuisine_encoder.fit_transform(df['Cuisines'].astype(str))

# Fill missing values and scale numerical columns
df.fillna(df.median(numeric_only=True), inplace=True)
scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Task 1: Predict Restaurant Ratings
features = numerical_cols + list(ohe.get_feature_names_out())
X = df[features]
y = df['Aggregate rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Task 1 - Restaurant Rating Prediction")
print(f"MSE: {mean_squared_error(y_test, y_pred)}")
print(f"R2 Score: {r2_score(y_test, y_pred)}")

# Task 2: Restaurant Recommendation using Nearest Neighbors
recommender = NearestNeighbors(n_neighbors=5, metric='euclidean')
recommender.fit(X)
def recommend_restaurants(index):
    distances, indices = recommender.kneighbors(X.iloc[[index]])  # Pass DataFrame slice with feature names
    recommendations = df.iloc[indices[0]][['Restaurant Name', 'Cuisines', 'City']]  # Show original 'Cuisines'
    return recommendations

print("Task 2 - Sample Recommendations:")
print(recommend_restaurants(0))

# Task 3: Cuisine Classification (Simplified)
# Group cuisines into broader categories (example: top 10 cuisines + 'Other')
top_cuisines = df['Cuisines'].value_counts().head(10).index
df['Cuisines_grouped'] = df['Cuisines'].apply(lambda x: x if x in top_cuisines else 'Other')
cuisine_group_encoder = LabelEncoder()
df['Cuisines_grouped_encoded'] = cuisine_group_encoder.fit_transform(df['Cuisines_grouped'])

X_class = df[features]
y_class = df['Cuisines_grouped_encoded']
X_train, X_test, y_train, y_test = train_test_split(X_class, y_class, test_size=0.2, random_state=42)
classifier = RandomForestClassifier(random_state=42)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print("Task 3 - Cuisine Classification (Grouped)")
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred, target_names=cuisine_group_encoder.classes_))

# Task 4: Geographical Analysis (unchanged)
map = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=10)
for _, row in df.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], popup=row['Restaurant Name']).add_to(map)
map.save("restaurants_map.html")
print("Task 4 - Geographical Analysis: Map saved as restaurants_map.html")

Task 1 - Restaurant Rating Prediction
MSE: 0.029558989494944996
R2 Score: 0.9852060454141679
Task 2 - Sample Recommendations:
       Restaurant Name                                Cuisines           City
0     Le Petit Souffle              French, Japanese, Desserts    Makati City
17          Balay Dako                                Filipino  Tagaytay City
9             Locavore                                Filipino     Pasig City
1837        feel ALIVE  North Indian, American, Asian, Biryani        Gurgaon
1     Izakaya Kikufuji                                Japanese    Makati City
Task 3 - Cuisine Classification (Grouped)
Accuracy: 0.6058890147225368
                                precision    recall  f1-score   support

                        Bakery       0.09      0.05      0.07        19
              Bakery, Desserts       0.00      0.00      0.00        18
                          Cafe       0.27      0.20      0.23        30
                       Chinese       0.00     